In [51]:
import sys
sys.path.append('./src')
import utils
from copy import deepcopy
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd


In [52]:
X_TRAIN_PATH = "data/interim/X_train.pkl"
y_TRAIN_PATH = "data/interim/y_train.pkl"
X_TEST_PATH = "data/interim/X_test.pkl"
y_TEST_PATH = "data/interim/y_test.pkl"
X_VALID_PATH = "data/interim/X_valid.pkl"
y_VALID_PATH = "data/interim/y_valid.pkl"


In [53]:
X_train = utils.deserialize_data(X_TRAIN_PATH)
y_train = utils.deserialize_data(y_TRAIN_PATH)
X_test = utils.deserialize_data(X_TEST_PATH)
y_test = utils.deserialize_data(y_TEST_PATH)
X_valid = utils.deserialize_data(X_VALID_PATH)
y_valid = utils.deserialize_data(y_VALID_PATH)


In [54]:
def drop_duplicate_data(X: pd.DataFrame, y: pd.Series) -> tuple:
    """
    Function to remove duplicate rows from a dataset.
    
    Parameters:
    X : pd.DataFrame
        The dataset from which to remove duplicate rows. Must be of type DataFrame.
    y : pd.Series
        The target data corresponding to the dataset X. Must be of type Series.
    
    Returns:
    tuple:
        A tuple containing:
        - X (pd.DataFrame): The dataset after duplicate rows have been dropped.
        - y (pd.Series): The target data aligned with the dataset after duplicates are removed.
    """
    if not isinstance(X, pd.DataFrame):
        raise TypeError("Parameter X must be of type DataFrame.")
    if not isinstance(y, pd.Series):
        raise TypeError("Parameter y must be of type Series.")
    
    print("Fungsi drop_duplicate_data: parameter telah divalidasi.")
    
    X = X.copy()
    y = y.copy()

    print(f"Fungsi drop_duplicate_data: shape dataset sebelum dropping duplicate adalah {X.shape}.")
    
    X_duplicate = X[X.duplicated()]
    print(f"Fungsi drop_duplicate_data: shape dari data yang duplicate adalah {X_duplicate.shape}.")
    
    X_clean = (X.shape[0] - X_duplicate.shape[0], X.shape[1])
    print(f"Fungsi drop_duplicate_data: shape dataset setelah drop duplicate seharusnya adalah {X_clean}.")
    
    X.drop_duplicates(inplace=True)
    
    y = y[X.index]
    
    print(f"Fungsi drop_duplicate_data: shape dataset setelah dropping duplicate adalah {X.shape}.")
    
    return X, y

In [55]:
X_train, y_train = drop_duplicate_data(X_train, y_train)

Fungsi drop_duplicate_data: parameter telah divalidasi.
Fungsi drop_duplicate_data: shape dataset sebelum dropping duplicate adalah (26064, 11).
Fungsi drop_duplicate_data: shape dari data yang duplicate adalah (96, 11).
Fungsi drop_duplicate_data: shape dataset setelah drop duplicate seharusnya adalah (25968, 11).
Fungsi drop_duplicate_data: shape dataset setelah dropping duplicate adalah (25968, 11).


In [56]:
def median_imputation(data: pd.DataFrame, subset_data, fit: bool) -> dict:
    """
    Impute missing values in specified columns of a DataFrame using the median.

    Parameters:
    - data: The DataFrame containing the data to be imputed.
    - subset_data: 
        - If fit is True: a list of column names to calculate medians.
        - If fit is False: a dictionary where keys are column names and values are their median values.
    - fit: A boolean indicating whether to fit the model (True) or perform imputation (False).

    Returns:
    - If fit is True, returns a dictionary of median values for the specified columns.
    - If fit is False, returns the DataFrame with missing values imputed.
    """

    if not isinstance(data, pd.DataFrame):
        raise RuntimeError("Fungsi median_imputation: parameter data haruslah bertipe DataFrame!")

    if fit:
        if not isinstance(subset_data, list):
            raise RuntimeError("Fungsi median_imputation: untuk nilai parameter fit = True, subset_data harus bertipe list dan berisi daftar nama kolom yang ingin dicari nilai mediannya guna menjadi data imputasi pada kolom tersebut.")
        imputation_data = {}
        for subset in subset_data:
            imputation_data[subset] = data[subset].median()

        print(f"Fungsi median_imputation: proses fitting telah selesai, berikut hasilnya {imputation_data}.")
        return imputation_data
    
    else:
        if not isinstance(subset_data, dict):
            raise RuntimeError("Fungsi median_imputation: untuk nilai parameter fit = False, subset_data harus bertipe dict dan berisi key yang merupakan nama kolom beserta value yang merupakan nilai median dari kolom tersebut.")

        print("Fungsi median_imputation: informasi count na sebelum dilakukan imputasi:")
        print(data.isna().sum())
        print()

        data.fillna(value=subset_data, inplace=True)

        print("Fungsi median_imputation: informasi count na setelah dilakukan imputasi:")
        print(data.isna().sum())
        print()

        return data

In [57]:
subset_data = ['person_emp_length', 'loan_int_rate']

subset_data = median_imputation(X_train, subset_data, fit=True)

X_train = median_imputation(X_train, subset_data, fit=False)

X_test = median_imputation(X_test, subset_data, fit=False)

X_valid = median_imputation(X_valid, subset_data, fit=False)

print("X_train setelah imputasi:")
print(X_train)
print("\nX_test setelah imputasi:")
print(X_test)
print("\nX_valid setelah imputasi:")
print(X_valid)

Fungsi median_imputation: proses fitting telah selesai, berikut hasilnya {'person_emp_length': np.float64(4.0), 'loan_int_rate': np.float64(10.99)}.
Fungsi median_imputation: informasi count na sebelum dilakukan imputasi:
person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length              734
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 2491
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
dtype: int64

Fungsi median_imputation: informasi count na setelah dilakukan imputasi:
person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_

In [58]:
from utils import serialize_data
def create_onehot_encoder(categories: list, path: str) -> OneHotEncoder:
    """
    Create and fit a OneHotEncoder with the specified categories, then save it to the given path.

    Parameters:
    - categories: A list of categories for which the encoder will be created.
    - path: The location on the disk where the encoder will be saved.

    Returns:
    - ohe: The fitted OneHotEncoder instance.
    """
    
    if not isinstance(categories, list):
        raise RuntimeError("Fungsi create_onehot_encoder: parameter categories haruslah bertipe list, berisi kategori yang akan dibuat encodernya.")

    if not isinstance(path, str):
        raise RuntimeError("Fungsi create_onehot_encoder: parameter path haruslah bertipe string, berisi lokasi pada disk komputer dimana encoder akan disimpan.")
    
    ohe = OneHotEncoder(sparse_output=False)  

    ohe.fit(np.array(categories).reshape(-1, 1))

    serialize_data(ohe, path)

    print(f"Kategori yang telah dipelajari adalah {ohe.categories_[0].tolist()}")

    return ohe

In [59]:
person_home_ownership = ['MORTGAGE', 'RENT', 'OWN', 'OTHER']
loan_intent = ['EDUCATION', 'PERSONAL', 'MEDICAL', 'VENTURE', 'HOMEIMPROVEMENT', 'DEBTCONSOLIDATION']
loan_grade = ['A', 'B', 'C', 'D', 'F', 'E', 'G']
cb_person_default_on_file = ['N', 'Y']

ohe_home_ownership = create_onehot_encoder(person_home_ownership, 'models/ohe_home_ownership.pkl')
ohe_loan_intent = create_onehot_encoder(loan_intent, 'models/ohe_loan_intent.pkl')
ohe_loan_grade = create_onehot_encoder(loan_grade, 'models/ohe_loan_grade.pkl')
ohe_default_on_file = create_onehot_encoder(cb_person_default_on_file, 'models/ohe_default_on_file.pkl')

Kategori yang telah dipelajari adalah ['MORTGAGE', 'OTHER', 'OWN', 'RENT']
Kategori yang telah dipelajari adalah ['DEBTCONSOLIDATION', 'EDUCATION', 'HOMEIMPROVEMENT', 'MEDICAL', 'PERSONAL', 'VENTURE']
Kategori yang telah dipelajari adalah ['A', 'B', 'C', 'D', 'E', 'F', 'G']
Kategori yang telah dipelajari adalah ['N', 'Y']


In [60]:
def ohe_transform(dataset: pd.DataFrame, subset: str, prefix: str, ohe: OneHotEncoder) -> pd.DataFrame:
    """
    Perform one-hot encoding on the specified subset of the dataset.

    Parameters:
    - dataset: The DataFrame containing the data to be encoded.
    - subset: The name of the column to be encoded.
    - prefix: The prefix to be added to the encoded columns.
    - ohe: The fitted OneHotEncoder instance.

    Returns:
    - dataset: The DataFrame with the encoded columns added and the original subset column dropped.
    """

    if not isinstance(dataset, pd.DataFrame):
        raise RuntimeError("Fungsi ohe_transform: parameter dataset harus bertipe DataFrame!")

    if not isinstance(ohe, OneHotEncoder):
        raise RuntimeError("Fungsi ohe_transform: parameter ohe harus bertipe OneHotEncoder!")

    if not isinstance(prefix, str):
        raise RuntimeError("Fungsi ohe_transform: parameter prefix harus bertipe str!")
    
    if not isinstance(subset, str):
        raise RuntimeError("Fungsi ohe_transform: parameter subset harus bertipe str!")

    try:
        col_names = list(dataset.columns)
        _ = col_names.index(subset) 
    except ValueError:
        raise RuntimeError("Fungsi ohe_transform: parameter subset string namun data tidak ditemukan dalam daftar kolom yang terdapat pada parameter dataset.")

    print("Fungsi ohe_transform: parameter telah divalidasi.")

    dataset = dataset.copy()

    print(f"Fungsi ohe_transform: daftar nama kolom sebelum dilakukan pengkodean adalah {list(dataset.columns)}")

    col_names = [f"{prefix}_{col_name}" for col_name in ohe.categories_[0].tolist()]

    encoded = pd.DataFrame(ohe.transform(dataset[[subset]]), columns=col_names, index=dataset.index)

    dataset = pd.concat([dataset, encoded], axis=1)

    dataset.drop(columns=[subset], inplace=True)

    print(f"Fungsi ohe_transform: daftar nama kolom setelah dilakukan pengkodean adalah {list(dataset.columns)}")

    return dataset

In [61]:
X_train = ohe_transform(X_train, "person_home_ownership", "home_ownership", ohe_home_ownership)
X_train = ohe_transform(X_train, "loan_intent", "loan_intent", ohe_loan_intent)
X_train = ohe_transform(X_train, "loan_grade", "loan_grade", ohe_loan_grade)
X_train = ohe_transform(X_train, "cb_person_default_on_file", "default_onfile", ohe_default_on_file)

X_test = ohe_transform(X_test, "person_home_ownership", "home_ownership", ohe_home_ownership)
X_test = ohe_transform(X_test, "loan_intent", "loan_intent", ohe_loan_intent)
X_test = ohe_transform(X_test, "loan_grade", "loan_grade", ohe_loan_grade)
X_test = ohe_transform(X_test, "cb_person_default_on_file", "default_onfile", ohe_default_on_file)

X_valid = ohe_transform(X_valid, "person_home_ownership", "home_ownership", ohe_home_ownership)
X_valid = ohe_transform(X_valid, "loan_intent", "loan_intent", ohe_loan_intent)
X_valid = ohe_transform(X_valid, "loan_grade", "loan_grade", ohe_loan_grade)
X_valid = ohe_transform(X_valid, "cb_person_default_on_file", "default_onfile", ohe_default_on_file)

Fungsi ohe_transform: parameter telah divalidasi.
Fungsi ohe_transform: daftar nama kolom sebelum dilakukan pengkodean adalah ['person_age', 'person_income', 'person_home_ownership', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length']
Fungsi ohe_transform: daftar nama kolom setelah dilakukan pengkodean adalah ['person_age', 'person_income', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length', 'home_ownership_MORTGAGE', 'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT']
Fungsi ohe_transform: parameter telah divalidasi.
Fungsi ohe_transform: daftar nama kolom sebelum dilakukan pengkodean adalah ['person_age', 'person_income', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file',

d:\software\New folder\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
d:\software\New folder\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
d:\software\New folder\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
d:\software\New folder\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
d:\software\New folder\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
d:\software\New folder\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
d:\software\New 

In [62]:
serialize_data(X_train, "data/processed/X_train_prep.pkl")

serialize_data(X_test, "data/processed/X_test_prep.pkl")

serialize_data(X_valid, "data/processed/X_valid_prep.pkl")


In [63]:
serialize_data(y_train, "data/processed/y_train_prep.pkl")